In [ ]:
%pip install beautifulsoup4

In [6]:
# Import the necessary libraries
## Extracing and processing country indicators from https://data.un.org/en/index.html
import requests #for extracting web content
import pandas as pd, numpy as np #for data manipulation
from bs4 import BeautifulSoup #for parsing web content

In [7]:
#import country names data
country_names = pd.read_csv("Countries1.csv")

#extract alphas column, converting values to lower case
country_codes = country_names['Alpha2'].str.lower()

In [30]:
#define target fields to extract
TARGET_FIELDS = {
    "Region": "region",
    "Population (000, 2025)": "population_th",
    "GDP: Gross domestic product (million current US$)": "GDP",
    "Life expectancy at birth (females/males, years)": "life_expectancy"
}

In [38]:
def extract_target_indicators(tables):
    
    result = {
        "region": None,
        "population_th": None,
        "gdp": None,
        "life_expectancy": None
    }
    
    # ---- GENERAL INFO TABLE ----
    general_rows = tables[0].find_all("tr")
    
    for row in general_rows:
        cols = [td.get_text(strip=True) for td in row.find_all("td")]
        if len(cols) < 3:
            continue
        
        label = cols[0].lower()
        value = cols[2]
        
        if "region" in label:
            result["region"] = value
            
        if "population" in label:
            result["population_th"] = value
    
    # ---- ECONOMIC TABLE ----
    econ_rows = tables[1].find_all("tr")
    
    for row in econ_rows:
        cols = [td.get_text(strip=True) for td in row.find_all("td")]
        if len(cols) < 4:
            continue
        
        label = cols[0].lower()
        
        if "gross domestic product" in label:
            result["gdp"] = cols[2]   # choose 2020 column
    
    # ---- SOCIAL TABLE ----
    social_rows = tables[2].find_all("tr")
    
    for row in social_rows:
        cols = [td.get_text(strip=True) for td in row.find_all("td")]
        if len(cols) < 4:
            continue
        
        label = cols[0].lower()
        
        if "life expectancy" in label:
            result["life_expectancy"] = cols[2]   # choose 2020
    
    return result


In [34]:
def extract_tables(url):
    headers = {"User-Agent": "Mozilla/5.0"}
    
    try:
        response = requests.get(url, headers=headers, timeout=10)
        
        if response.status_code != 200:
            print(f"Skipping (not found): {url}")
            return None
        
        soup = BeautifulSoup(response.text, "html.parser")
        tables = soup.find_all("table", class_="pure-table")
        
        return tables
    
    except Exception as e:
        print(f"Error loading {url}: {e}")
        return None


In [39]:
all_country_data = []

all_country_data = []

for country_name, iso_code in zip(
        country_names["Country"].iloc[:10],
        country_names["Alpha2"].iloc[:10].astype(str).str.lower()):
    
    url = f"https://data.un.org/en/iso/{iso_code}.html"
    
    print(f"Scraping {country_name}...")
    
    tables = extract_tables(url)
    
    if not tables:
        continue
    
    indicators = extract_target_indicators(tables)
    
    indicators["country"] = country_name
    indicators["iso_alpha2"] = iso_code.upper()
    
    all_country_data.append(indicators)

all_country_data_df = pd.DataFrame(all_country_data)

Scraping Afghanistan...
Scraping Albania...
Scraping Algeria...
Scraping American Samoa...
Scraping Andorra...
Scraping Angola...
Scraping Anguilla...
Scraping Antigua and Barbuda...
Scraping Argentina...
Scraping Armenia...


In [40]:
print(all_country_data_df.head())

            region population_th      gdp    life_expectancy         country  \
0    Southern Asia       43 844a   19 983  64.7 /       58.5     Afghanistan   
1  Southern Europe        2 772a   15 163  80.3 /       75.4         Albania   
2  Northern Africa       47 435a  158 975  74.9 /       71.7         Algeria   
3        Polynesia         46a,b     None  75.7 /       70.0  American Samoa   
4  Southern Europe           83a    2 891  83.8 /       75.8         Andorra   

  iso_alpha2  
0         AF  
1         AL  
2         DZ  
3         AS  
4         AD  


In [ ]:
#Clean and save the data
import re

all_country_data_df["population_th"] = (
    all_country_data_df["population_th"]
    .astype(str)
    .str.replace(" ", "", regex=False)
    .str.replace(r"[^0-9]", "", regex=True)
)

all_country_data_df["population_th"] = pd.to_numeric(
    all_country_data_df["population_th"],
    errors="coerce"
)

all_country_data_df["gdp"] = (
    all_country_data_df["gdp"]
    .astype(str)
    .str.replace(" ", "", regex=False)
    .str.replace(r"[^0-9.-]", "", regex=True)
)

all_country_data_df["gdp"] = pd.to_numeric(
    all_country_data_df["gdp"],
    errors="coerce"
)

life_split = all_country_data_df["life_expectancy"].str.split("/", expand=True)

all_country_data_df["life_expectancy_female"] = pd.to_numeric(
    life_split[0].str.replace(r"[^0-9.]", "", regex=True),
    errors="coerce"
)

all_country_data_df["life_expectancy_male"] = pd.to_numeric(
    life_split[1].str.replace(r"[^0-9.]", "", regex=True),
    errors="coerce"
)
